# Navigation

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

In [1]:
!pip -q install ./python

tensorflow 1.7.1 has requirement numpy>=1.13.3, but you'll have numpy 1.12.1 which is incompatible.
ipython 6.5.0 has requirement prompt-toolkit<2.0.0,>=1.0.15, but you'll have prompt-toolkit 3.0.31 which is incompatible.
jupyter-console 6.4.3 has requirement jupyter-client>=7.0.0, but you'll have jupyter-client 5.2.4 which is incompatible.


The environment is already saved in the Workspace and can be accessed at the file path provided below.  Please run the next code cell without making any changes.

In [ ]:
from unityagents import UnityEnvironment
import random
from collections import namedtuple, deque
import numpy as np
from pathlib import Path
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

### 2. Create an Environment Wrapper Class

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

The following classes will create an environment wrapper whose methods will factor in the concept of "brains" that exists in the Unity environment.

In [ ]:
class RLEnv:
    def __init__(self):
        self.n_actions: int = None
        self.n_states: int = None
        
    def reset(self) -> None:
        pass
    
    def step(self, action) -> None:
        pass
    

class BananaEnvironment(RLEnv):
    def __init__(self, fp:str):
        self.unity = UnityEnvironment(file_name=fp)
        
        # Get the default brain
        self.brain_name = self.unity.brain_names[0]
        self.brain = self.unity.brains[self.brain_name]
        
        # Get action space
        self.n_actions = self.brain.vector_action_space_size
        
        # Get state space
        env_info = self.unity.reset(train_mode=True)[self.brain_name]
        self.n_states = len(env_info.vector_observations[0])
        
    def reset(self):
        env_info = self.unity.reset(train_mode=True)[self.brain_name]
        return env_info.vector_observations[0]
        
    def step(self, action):
        env_info = self.unity.step(action)[self.brain_name]
        next_state = env_info.vector_observations[0]
        reward = env_info.rewards[0]
        done = env_info.local_done[0]
        return next_state, reward, done
        
    def close(self):
        self.unity.close()

### 3. Create the Q-Network Architecture

We will create the Q-network model architecture. This will be a simple feed-forward neural network.

In [ ]:
class QNetwork(nn.Module):
    """Actor (Policy) Model."""

    def __init__(self, state_size, action_size, seed, fc1_units=64, fc2_units=64):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            seed (int): Random seed
            fc1_units (int): Number of nodes in first hidden layer
            fc2_units (int): Number of nodes in second hidden layer
        """
        super(QNetwork, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(state_size, fc1_units)
        self.fc2 = nn.Linear(fc1_units, fc2_units)
        self.fc3 = nn.Linear(fc2_units, action_size)

    def forward(self, state):
        """Build a network that maps state -> action values."""
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

Before we train the neural network, we will first make sure to establish a connection to the GPU device if we have available.

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### 4. Create the Replay Buffer Class

We will next create the replay buffer class.

In [ ]:
class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, action_size, buffer_size, batch_size, seed):
        """Initialize a ReplayBuffer object.

        Params
        ======
            action_size (int): dimension of each action
            buffer_size (int): maximum size of buffer
            batch_size (int): size of each training batch
            seed (int): random seed
        """
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)  
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
        self.seed = random.seed(seed)
    
    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)
    
    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=self.batch_size)

        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).long().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)
  
        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)

### 5. Create the Agent Class

We will be setting up an agent that hosts a DQN model and learns from input experiences.

In [ ]:
class Agent:
    """Interacts with and learns from the environment."""

    def __init__(self, env:RLEnv, seed, lr:float, buffer_size:int,
                 batch_size:int, update_every:int, gamma:float, tau:float):
        """Initialize an Agent object.
        
        Params
        ======
            env (RLEnv): the environment
            seed (int): random seed
            lr (float): learning rate of the DQN model
            buffer_size (int): replay buffer size
            batch_size (int): minibatch size for model training
            update_every (int): how often to update the network
            gamma (float): reward discount factor
            tau (float): constant for soft update of target parameters
        """
        self.env = env
        self.seed = random.seed(seed)
        
        # Hyperparameters
        self.lr = lr
        self.buffer_size = buffer_size
        self.batch_size = batch_size
        self.update_every = update_every
        self.gamma = gamma
        self.tau = tau

        # Q-Network
        self.qnetwork_local = QNetwork(self.env.n_states, self.env.n_actions, seed).to(device)
        self.qnetwork_target = QNetwork(self.env.n_states, self.env.n_actions, seed).to(device)
        self.optimizer = torch.optim.Adam(self.qnetwork_local.parameters(), lr=self.lr)

        # Replay memory
        self.memory = ReplayBuffer(action_size=self.env.n_actions, 
                                   buffer_size=self.buffer_size, 
                                   batch_size=self.batch_size, 
                                   seed=seed)
        
        # Initialize time step (for updating every UPDATE_EVERY steps)
        self.t_step = 0
    
    def step(self, state, action, reward, next_state, done):
        # Save experience in replay memory
        self.memory.add(state, action, reward, next_state, done)
        
        # Learn every UPDATE_EVERY time steps.
        self.t_step = (self.t_step + 1) % self.update_every
        if self.t_step == 0:
            
            # If enough samples are available in memory, get random subset and learn
            if len(self.memory) > self.batch_size:
                experiences = self.memory.sample()
                self.learn(experiences)

    def act(self, state, eps=0.):
        """Returns actions for given state as per current policy.
        
        Params
        ======
            state (array_like): current state
            eps (float): epsilon, for epsilon-greedy action selection
        """
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        self.qnetwork_local.eval()
        with torch.no_grad():
            action_values = self.qnetwork_local(state)
        self.qnetwork_local.train()

        # Epsilon-greedy action selection
        if random.random() > eps:
            return np.argmax(action_values.cpu().data.numpy())
        else:
            return random.choice(np.arange(self.env.n_actions))

    def learn(self, experiences):
        """Update value parameters using given batch of experience tuples.

        Params
        ======
            experiences (Tuple[torch.Tensor]): tuple of (s, a, r, s', done) tuples 
            gamma (float): discount factor
        """
        states, actions, rewards, next_states, dones = experiences

        # Get max predicted Q values (for next states) from target model
        Q_targets_next = self.qnetwork_target(next_states).detach().max(1)[0].unsqueeze(1)
        
        # Compute Q targets for current states 
        Q_targets = rewards + (self.gamma * Q_targets_next * (1 - dones))

        # Get expected Q values from local model
        Q_expected = self.qnetwork_local(states).gather(1, actions)

        # Compute loss
        loss = F.mse_loss(Q_expected, Q_targets)
        
        # Minimize the loss
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # ------------------- update target network ------------------- #
        self.soft_update(local_model=self.qnetwork_local, 
                         target_model=self.qnetwork_target)                     

    def soft_update(self, local_model, target_model):
        """Soft update model parameters.
        θ_target = τ*θ_local + (1 - τ)*θ_target

        Params
        ======
            local_model (PyTorch model): weights will be copied from
            target_model (PyTorch model): weights will be copied to
            tau (float): interpolation parameter 
        """
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(self.tau * local_param.data + (1.0 - self.tau) * target_param.data)
            
    def train_dqn(self, n_episodes=2000, max_t=1000, eps_start=1.0, 
                  eps_end=0.01, eps_decay=0.995, plot=True) -> None:
        """Deep Q-Learning.
    
        Params
        ======
            n_episodes (int): maximum number of training episodes
            max_t (int): maximum number of timesteps per episode
            eps_start (float): starting value of epsilon, for epsilon-greedy action selection
            eps_end (float): minimum value of epsilon
            eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
        """
        scores = []                        # list containing scores from each episode
        scores_window = deque(maxlen=100)  # last 100 scores
        eps = eps_start                    # initialize epsilon
        
        # Iterate through epochs
        for i_episode in range(1, n_episodes+1):
            state = self.env.reset()
            score = 0
            
            # Iterate through timesteps until we reach a "done" state
            for t in range(max_t):
                action = self.act(state, eps)
                next_state, reward, done = self.env.step(action)
                self.step(state, action, reward, next_state, done)
                state = next_state
                score += reward
                
                # Marks the end of an episode
                if done:
                    break
                
            # Append to score window
            scores_window.append(score)       # save most recent score
            scores.append(score)              # save most recent score
            
            # Epsilon decay
            eps = max(eps_end, eps_decay*eps)
            
            # Print episode statistics
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
            if i_episode % 100 == 0:
                print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
            if np.mean(scores_window) >= 13.0:
                print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
                self.save_model("checkpoint.pth")
                break
        
        # Plot scores if needed
        if plot:
            fig = plt.figure()
            ax = fig.add_subplot(111)
            plt.plot(np.arange(len(scores)), scores)
            plt.ylabel('Score')
            plt.xlabel('Episode #')
            plt.show()
    
    def save_model(self, fp:str) -> None:
        torch.save(self.qnetwork_local.state_dict(), fp)
    
    def load_model(self, fp:str) -> None:
        self.qnetwork_local.load_state_dict(torch.load(fp))

### 6. Train a DQN on the Banana Environment

We will now initialize a DQN Agent and run it on the Banana environment. Results will be plotted after the "train_dqn" method is called.

In [ ]:
# Create environment
env = BananaEnvironment(fp="/data/Banana_Linux_NoVis/Banana.x86_64")

# Create agent
agent = Agent(env=env,
            seed=0,
            lr=5e-4,
            buffer_size=100000,
            batch_size=64,
            update_every=4,
            gamma=0.99,
            tau=1e-3)

# Train agent
agent.train_dqn(n_episodes=1000,
                max_t=1000,
                eps_start=1,
                eps_end=0.01,
                eps_decay=0.995)

# Close environment
env.close()